# Case_Study

## Summary:
    
#### I have processed a cab trip dataset, conducting data cleaning and employing fundamental Exploratory Data Analysis (EDA) techniques. Through analysis of features such as customer_request_id, pick-up point, driver id, status, request timestamp, and drop timestamp, I derived broader insights and conclusions..

NOTE: This is just an exercise for educational purpose.

#### Importing the required libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## 1. Facts and Figures about our dataset(Understanding the data)

#### Read the csv into a dataframe(df) and print the first 5 rows

In [ ]:
df=pd.read_csv("../input/uber-cab-request-data/Uber Request Data.csv.xls")
df.head()

In [ ]:
len(df["Request id"].unique())

#### Show the number of rows and columns in the dataset

In [ ]:
df.shape

#### Show the number of NaNs in each columns

In [ ]:
df.isnull().sum()

#### Show the % of NaNs in each columns

In [ ]:
df.isnull().sum()/df.shape[0]*100 # df.shape[0] gives the number of rows

#### Show the details ssuch as column names,dtypes,non-null count for the given data

#### Extracted Info:
##### Number of Rows: 6745
##### Number of Columns: 6
##### Dtype of each column based on the type of data it holds
##### There are 2 numerical(int and float) anf 4 objects columns

In [ ]:
df.info()

#### Finding out the statistical summary for both numerical and object columns in one place

In [ ]:
df.describe(include="all")

## 2. Cleaning/Handling the data

#### We see that for the column "Request timstamp", the format of DateTime is different for different sections of data. Some have it separated by "-" and some have it separated by "/". 
#### Let us first replace the "/" with "-" to have uniformity and then convert the enite column to standard datatime format using "pd.to_datatime" function. Refer to lines 10,11 and 12.

In [ ]:
df["Request timestamp"].value_counts()

In [ ]:
df["Request timestamp"]=df["Request timestamp"].astype(str)

In [ ]:
df["Request timestamp"] = df["Request timestamp"].apply(lambda x: x.replace('/','-'))

In [ ]:
df["Request timestamp"]=pd.to_datetime(df["Request timestamp"],dayfirst=True,format='mixed')

#### After doing the datatime conversion, if we pull the info again, we see that "Request timestamp" is converted to "datetime".

In [ ]:
df.info()

#### Similarly convert "Drop timestamp" column from "object" to "datatime"

In [ ]:
df["Drop timestamp"]=pd.to_datetime(df["Drop timestamp"],dayfirst=True,format='mixed')

#### Pulling up the info, now we can see both "Request timestamp" and "Drop timestamp" are converted to datatime standard.

In [ ]:
df.info()

In [ ]:
df["Drop timestamp"]

#### We now add 2 columns "req_hour"(which is Hour of the request during the day) and "req_day"(which is the day of the month) to determine and catogorise the load of cab service requests.

In [ ]:
df["req_hour"]=df["Request timestamp"].dt.hour # Fetching the hour number from the request timestamp and Adding a new column "req_hour"

In [ ]:
df["req_day"]=df["Request timestamp"].dt.day # Fetching the day number from the request timestamp and Adding a new column "req_day"

## Note: 
#### The NaNs/missing values in the column "Driver_Id" can be ignored.
#### This is because we see that since there were NO CARS AVAILABLE at point of the day after the user tried to book a cab, no driver was alloted the trip and hence the driver_id is empty.
#### Similary, we can ignore the NaNs/missing values in the column "Drop timestamp" as for all of them, the trip is either CANCELLED or NO CARS AVAILABLE.
#### In both the above cases, the data is missing due to a genuine reason and not that it got lost during data collection. Hence, I have not substitued them with any other values based on any logic. 

In [ ]:
sns.countplot(x="req_hour",data=df,hue="Status")
plt.show()

# 3. Visualize and Analyse

In [ ]:
sns.catplot(x="req_hour",data=df,hue="Pickup point",kind="count")
plt.show()

#### Plot of Status of the trip at different hours of the day and also pick up locations shows that,
#### 1) Between hours 5AM-9AM, the load on cabs are high with almost equal amount of trips getting completed and cancelled.
#### 2) Between hours 5PM-9PM, the load on cabs are significantly high. Hence, there is mismatch between cab demand and availabilty. Hence, we see more of "No cars Available Status".
#### 3) Between hours 5AM-9AM, the users from city is significantly high.
#### 4) Between hours 5PM-9PM, the users from Airport is significantly high.

#### Adding a new column Time_Slot to make categories of hours from the req_hour column

In [ ]:
df["Time_Slot"]=0

In [ ]:
df.head()

### Hour Categories:
    
#### req_hour<5 as "Pre_Morning"
#### req_hour<10 as "Morning_Rush"
#### req_hour<17 as "Day_Time"
#### req_hour<22 as "Evening_Rush"
#### else "Late_Night"

In [ ]:
j=0
for i in df["req_hour"]:
    if df.loc[j,'req_hour']<5:
        df.loc[j,'Time_Slot']="Pre_Morning"
    elif 5<=df.loc[j,'req_hour']<10:
        df.loc[j,'Time_Slot']="Morning_Rush"
        
    elif 10<=df.loc[j,'req_hour']<17:
        df.loc[j,'Time_Slot']="Day_Time"
        
    elif 17<=df.loc[j,'req_hour']<22:
        df.loc[j,'Time_Slot']="Evening_Rush"
    else:
        df.loc[j,'Time_Slot']="Late_Night"
    j=j+1

In [ ]:
df.head()

In [ ]:
df["Time_Slot"].value_counts()

#### You see from the above value counts, the "Morning_Rush" and "Evening_Rush" are the hours with maximum load.

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x="Time_Slot",hue="Status",data=df)
plt.show()

In [ ]:
df_morning_rush=df[df['Time_Slot']=='Morning_Rush']

In [ ]:
sns.countplot(x="Pickup point",hue="Status",data=df_morning_rush)

# Cancellation of cab as per the pickup location at morning rush hours

In [ ]:
# Pickup point= Airport

df_airport_cancelled=df_morning_rush.loc[(df_morning_rush["Pickup point"]=="Airport") & (df_morning_rush["Status"]=="Cancelled")]
len(df_airport_cancelled)

In [ ]:
# Pickup point= City
df_city_cancelled=df_morning_rush.loc[(df_morning_rush["Pickup point"]=="City") & (df_morning_rush["Status"]=="Cancelled")]
len(df_city_cancelled)

# Morning Rush- city

In [ ]:
df_morning_rush[(df_morning_rush["Pickup point"]=="City")].shape[0]

In [ ]:
df_morning_rush[(df_morning_rush["Pickup point"]=="City") & (df_morning_rush["Status"]=="Cancelled")].shape[0]

In [ ]:
df_morning_rush[(df_morning_rush["Pickup point"]=="City") & (df_morning_rush["Status"]=="Trip Completed")].shape[0]

In [ ]:
df_morning_rush[(df_morning_rush["Pickup point"]=="City") & (df_morning_rush["Status"]=="No Cars Available")].shape[0]

# Morning Rush- Airport

In [ ]:
df_morning_rush[(df_morning_rush["Pickup point"]=="Airport")].shape[0]

In [ ]:
df_morning_rush[(df_morning_rush["Pickup point"]=="Airport") & (df_morning_rush["Status"]=="Cancelled")].shape[0]

In [ ]:
df_morning_rush[(df_morning_rush["Pickup point"]=="Airport") & (df_morning_rush["Status"]=="Trip Completed")].shape[0]

In [ ]:
df_morning_rush[(df_morning_rush["Pickup point"]=="Airport") & (df_morning_rush["Status"]=="No Cars Available")].shape[0]

# Evening Rush

In [ ]:
df_evening_rush=df[df['Time_Slot']=='Evening_Rush']
df_evening_rush.head()

In [ ]:
df_city_cancelled=df_evening_rush[(df_evening_rush["Pickup point"]=="City") & (df_evening_rush["Status"]=="Cancelled")]

In [ ]:
sns.countplot(x="Pickup point",hue="Status",data=df_evening_rush)

In [ ]:
df_evening_rush["Status"].value_counts()

# Evening- Rush City

In [ ]:
df_evening_rush[(df_evening_rush["Pickup point"]=="City")].shape[0]

In [ ]:
df_evening_rush[(df_evening_rush["Pickup point"]=="City") & (df_evening_rush["Status"]=="Cancelled")].shape[0]

In [ ]:
df_evening_rush[(df_evening_rush["Pickup point"]=="City") & (df_evening_rush["Status"]=="Trip Completed")].shape[0]

In [ ]:
df_evening_rush[(df_evening_rush["Pickup point"]=="City") & (df_evening_rush["Status"]=="No Cars Available")].shape[0]

# Evening- Rush Airport

In [ ]:
df_evening_rush[(df_evening_rush["Pickup point"]=="Airport")].shape[0]

In [ ]:
df_evening_rush[(df_evening_rush["Pickup point"]=="Airport") & (df_evening_rush["Status"]=="Cancelled")].shape[0]

In [ ]:
df_evening_rush[(df_evening_rush["Pickup point"]=="Airport") & (df_evening_rush["Status"]=="Trip Completed")].shape[0]

In [ ]:
df_evening_rush[(df_evening_rush["Pickup point"]=="Airport") & (df_evening_rush["Status"]=="No Cars Available")].shape[0]

# Pie chart

In [ ]:
df_morning_city=df[(df["Pickup point"]=="City")&(df["Time_Slot"]=="Morning_Rush")]

In [ ]:
df_morning_city_count=pd.DataFrame(df_morning_city["Status"].value_counts())

In [ ]:
df_morning_city_count

In [ ]:
df_morning_city_count["count"].values

In [ ]:
df_morning_city_count.index

In [ ]:
fig,ax=plt.subplots()
ax.pie(df_morning_city_count["count"].values,labels=df_morning_city_count["count"].index,
      autopct="%.2f%%",startangle=90)
plt.show()

In [ ]:
df_evening_airport=df[(df["Pickup point"]=="Airport")&(df["Time_Slot"]=="Evening_Rush")]
df_evening_airport_count=pd.DataFrame(df_evening_airport["Status"].value_counts())
df_evening_airport_count

In [ ]:
df_evening_airport_count["count"].values

In [ ]:
df_evening_airport_count["count"].index

In [ ]:
fig,ax=plt.subplots()
ax.pie(df_evening_airport_count["count"].values,labels=df_evening_airport_count["count"].index,
      autopct="%.2f%%",startangle=90)
plt.show()

# Key Take Aways

#### 1) We understood the dataset with the number of user requests were done and the number of columns(6745,6) along with other facts such as number/percentage of NaNs in each columns and format of dataTime in the request and drop timestamp columns.
#### 2) We standardised format of dataTime in the request and drop timestamp columns.
#### 3) Figured a logical reason as to why the NaNs in Driver_id and drop timestamp columns should be ignored.
#### 4) Extracted the day number and hour from the request timestamp column to perform a deeper analysis.
#### 5) Between hours 5AM-9AM, the load on cabs are high with almost equal amount of trips getting completed and cancelled.
#### 6) Between hours 5PM-9PM, the load on cabs are significantly high. Hence, there is mismatch between cab demand and availabilty. Hence, we see more of "No cars Available Status".
#### 7) Between hours 5AM-9AM, the users from city is significantly high.
#### 8) Between hours 5PM-9PM, the users from Airport is significantly high.
#### 9) The "Morning_Rush" and "Evening_Rush" are the hours with maximum load(i.e more number of users requesting cab services).
#### 10) We also saw the load during "Morning_rush" and "Evening_Rush" from both pick-up points "City" and "Airport".
#### 11) During the evening rush hour, we saw a significant number of No CARS AVAILABLE status  for the trip bookings from "Airport".